![example](images/camera-filming-banner2.jpg)

# A Movie Production Analysis for Microsoft

**Authors:** Sameeha Ramadhan

## Overview

This analysis attempts to satisfy the informational needs of Microsoft (MSFT) by investigating the film industry to determine which types of films their new studio should produce. We will look at a number of factors to better understand what contributes to the success of a movie, and in turn offer a few recommendations on the best strategies to get started. Descriptive analysis of popular genres and ratings data show that while the preferred movie types that some people enjoy varies seasonally (think: horror films around Halloween and Christmas movies during the Christmas season), overall the three types of films that are currently doing the best are: . Microsoft can use this analysis to adjust planning, production, and marketing to hit the ground running as they enter this highly competitve space.


 # Business Problem
 ![example](images/data-film.jpg)

Microsoft may be able to enter the movie making sector and successfully stand out from fierce competition by choosing to create films that their target audiences have shown to thoroughly enjoy. Doing so will allow them to produce movies that will instantly become hits, which will in turn allow them to improve on and produce even more content, setting them up to be a studio force to be reckoned with. Using data from well-known industry sources such as Imdb, I analyze and explain patterns in popular movie types based on ratings and reviews, as well as budgeting decisions to help predict what audiences want from a film and thus, guaranteeing its success.

## Data Understanding

Rotten Tomatoes is a well-known review-aggregation website for television and film. Due to its frequent use by the American public, it is widely viewed as the 'go-to' in deciding whether or not a movie or show is worth watching. IMDB.com is a database that is considered the most authoritative source for all film and television related material. The data from both websites provides information such as, but not limited to, budgets, reviews and ratings.

***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline

In [2]:
#Provided datasets:
title_ratings = pd.read_csv('./data/imdb.title.ratings.csv.gz')
title_akas = pd.read_csv('./data/imdb.title.akas.csv.gz')
imdb_basics = pd.read_csv('./data/imdb.title.basics.csv.gz')
name_basics = pd.read_csv('./data/imdb.name.basics.csv.gz')
title_principals = pd.read_csv('./data/imdb.title.principals.csv.gz')
movies = pd.read_csv('./data/tmdb.movies.csv.gz')
movies_gross = pd.read_csv('./data/bom.movie_gross.csv.gz')
movies_budget = pd.read_csv('./data/tn.movie_budgets.csv.gz')
rt_critic_reviews = pd.read_csv('./data/rotten_tomatoes_critic_reviews.csv.gz')
rt_movies = pd.read_csv('./data/rotten_tomatoes_movies.csv.gz')

When starting this analysis, the first thought that comes to mind is what are the most popular types of movies and what are the top over profits for the top 3 genres. To start exploring these facts, I will begin by examining and sorting the top 10 movie genres, and then plotting the results for easy viewing.

In [3]:
#Preview the imdb_basics table.
imdb_basics.head(5)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [4]:
#I make a new dataframe from the 'genres' column by first splitting the column where each comma is found. I then pivot
#the table using the explode method while counting the number of times each genre appears by using the value_counts method.
#Then, preview.
genre_count = imdb_basics['genres'].str.split(',').explode().value_counts()
movie_genres = pd.DataFrame(genre_count)
#Rename the columns
movie_genres.reset_index(inplace=True)
movie_genres = movie_genres.rename(columns = {'index':'Genre', 'genres':'Movie Count'})
movie_genres.head()

,Genre,Movie Count
0,Documentary,51640
1,Drama,49883
2,Comedy,25312
3,Thriller,11883
4,Horror,10805


I will first determine how much profit each movie has made. To do so, we start by looking at the movies_budget data frame.


In [ ]:
#Preview the movies_budget data frame.
movies_budget.head(5)

In [ ]:
#I make a new data frame so I can return to the original one should need be:
profitable_movies = movies_budget
#Then I will need to convert the production_budget, domestic_gross, and worldwide_gross columns to integers in order to 
#make calculations. First, I remove the '$':
profitable_movies.domestic_gross = [x.strip('$') for x in profitable_movies.domestic_gross]
profitable_movies.production_budget = [x.strip('$') for x in profitable_movies.production_budget]
profitable_movies.worldwide_gross = [x.strip('$') for x in profitable_movies.worldwide_gross]

In [ ]:
#Then, I remove the commas,
profitable_movies.domestic_gross = profitable_movies['domestic_gross'].astype(str).str.replace(',', '').astype(int) 
profitable_movies.production_budget = profitable_movies['production_budget'].astype(str).str.replace(',', '').astype(int) 
profitable_movies.worldwide_gross = profitable_movies['worldwide_gross'].astype(str).str.replace(',', '').astype(np.int64) 
#Because the numbers in this column are large, I used the .astype(np.int64) method.

Now let's remove all non profitable movies and have a look at our new data frame.

In [ ]:
profitable_movies = movies_budget[movies_budget['domestic_gross'] !=0]
profitable_movies

Next, I'll create and add a new column by calculating the profit of each film and defining it as profits.

In [ ]:
profitable_movies['profit'] = profitable_movies['worldwide_gross'] - profitable_movies['production_budget']
profitable_movies

And I'll add a profit margin column so we can really evaluate how much a movie has made:

In [ ]:
profitable_movies['profit_margin'] = (profitable_movies['worldwide_gross'] - 
                                    profitable_movies['production_budget'])/profitable_movies['worldwide_gross']
profitable_movies.head(10)

In [ ]:
profitable_movies.query('movie=="Dark Phoenix"')\

Immediately the movie Dark Phoenix stands out as having a negative profit and profit margin which means that the movie did not make money. Since we are trying to determine which movies did the best, I will remove all films with a negative margin and arrange them by profit, from most profitable to least.

In [ ]:
films_with_profit = profitable_movies.loc[profitable_movies['profit'] > 0]
movies_ranked = films_with_profit.sort_values(by=['profit'], ascending=False)
movies_ranked.reset_index(inplace=True)
movies_ranked.drop(columns=['index'], inplace=True)
movies_ranked.head(20)

In [ ]:
ax1 = sns.lmplot(x='production_budget', y='profit', data=movies_ranked, line_kws={'color': 'green'}, height=7, aspect=2)
plt.xlabel('Movie Budget (Hundreds of Millions of Dollars)', fontsize=10)
plt.ticklabel_format(axis='x')
plt.ylabel('Movie Profit (Billions of Dollars)', fontsize=10)
plt.title('Movie Budget vs Movie Profit', fontsize=12);

The upward trend (as indicated by the green line) shows that in general as a movie's budget increases, so does its profit. Based on this plot, we can begin to get a sense of what a budget should be per movie. To get a better look I will plot the top 20 movies and their budgets.

In [ ]:
plt.figure(figsize=(15,12))
sns.barplot(x=movies_ranked.loc[0:20, 'movie'],y=movies_ranked.loc[0:20, 'profit'], 
            color='green', label='Profit')
sns.barplot(x=movies_ranked.loc[0:20, 'movie'],y=movies_ranked.loc[0:20, 'production_budget'], 
            color='purple', label='Budget')
plt.xlabel('Movie', fontsize=12, color='blue')
plt.title("Profits and Budgets for the Top 20 Profitable Movies", fontsize=14)
plt.ylabel('Profit (Billions of Dollars)', fontsize=12, color='blue')
plt.xticks(rotation=25, horizontalalignment='right', fontsize=10)
plt.legend(fontsize=12);

We can see clearly that the top 20 movies have incredible profits, with a mean margin of a whopping 86%! This is valuable information, especially considering that an average amount of $207,360,000 was spent per production.

In [ ]:
movies_ranked.loc[0:20, 'production_budget'].mean()

In [ ]:
print(movies_ranked.loc[0:20, 'profit_margin'].mean())
print(movies_ranked.loc[0:20, 'production_budget'].mean())

Using the facts we've extracted above, we can see that the majority of the top 20 movies have a margin above 80% and now can decide on a set profit margin that we'd like to see.
Based on this, we could suggest that Microsoft aim to spend about $200,000,00 per movie with an expectation of at lease 80% in profits, however this conclusion could be flawed since many high budget movies have not had high profit margins. See the previous observation we've made that the movie Dark Phoenix not only did not earn any profit, but it also lost a large amount despite it's high budget:

In [ ]:
profitable_movies.query('movie=="Dark Phoenix"')

So to get a more accurate figure, I will examine all of the profitable movies, not simply the top 20. In doing so, I immediately notice that the production budget increased while the profits and profit margins decreased (by almost 20%!).

In [ ]:
movies_ranked.describe()

In [ ]:
movies_ranked['production_budget'].mean()

I've decided based on this information that to filter the data with a profit margin of at least 70% and a budget greater than $38,800,385 makes the most sense, I believe a margin any lower than 70% would not be the preferred aim of a new movie studio.

In [ ]:
profit_over_70 = profitable_movies.loc[(profitable_movies['profit_margin'] >= 0.70) & 
                         (profitable_movies['production_budget'] > 38800385)]
profit_over_70

And finally, I will plot this new data frame to examine if our theory that the larger the budget, the higher the profit, holds true:

In [ ]:
ax2 = sns.lmplot(x='production_budget', y='profit_margin', data=profit_over_70, 
                 line_kws={'color': 'red'}, height=7, aspect=2)
plt.xlabel('Movie Budget (Hundreds of Millions of Dollars)', fontsize=12, color='red')
plt.ticklabel_format(axis='x')
plt.ylabel('Profit Margin)', fontsize=12, color='red')
plt.title('Movie Budget vs Movie Profit', fontsize=12);

And with this visualization we can clearly see that our theory in fact did not hold true, since the trend line (indicated in red) is negative, and thus I'd recommend that Microsoft use careful restraint in budgeting. Larger budgets do not necessarily translate to larger profits, infact, it may lead to very low profit margins. In my final examination, I notice an average profit margin of and a budget of. **So to conclude: I recommend that Microsoft budget around $96,194,000 for production, which should produce a profit margin of at least 79%.**

In [ ]:
print(profit_over_70['production_budget'].mean())
print(profit_over_70['profit_margin'].mean())

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***